In [ ]:
import json
import pickle
import numpy as np
import time
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs
import os
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.rdmolfiles import SDWriter
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

### Unless you have your ligand files in separate .sdf files in the Ligand Folder select one of the following:

## Option 1: Ligand Molecules are in a single .sdf file (i.e. downloaded from PDB)

In [ ]:
single_sdf = 'Example_Lopinavir.sdf'
ligand_folder = 'Ligands'

#CREATE SDF FILES FOR THE FIRST TIME
suppl = Chem.SDMolSupplier('%s/%s' %(ligand_folder,single_sdf))
mols_op = [x for x in suppl]
mols_opID = [x.GetProp("ChemCompId") for x in mols_op]
#print(mols_opID)
uniques_molID = []
uniques_molID_indx = []
for j,i in enumerate(mols_opID):
    if i in uniques_molID:
        continue
    else:
        uniques_molID += [i]
        uniques_molID_indx += [j]
unique_sdfs = [mols_op[i] for i in uniques_molID_indx]
for j, query in enumerate(unique_sdfs):
    writer = SDWriter('%s/%s.sdf' %(ligand_folder, uniques_molID[j]))
    writer.write(query)

## Option 2: Ligand Molecules as SMILES strings

In [ ]:
smilesstrings = ["CC1(C)S[CH](N[CH]1C(O)=O)[CH](NC(=O)Cc2ccccc2)C(=O)NCc3ccccc3", "CC1(C)S[CH](N[CH]1C(=O)N[CH](CO)Cc2ccccc2)[CH](NC(=O)Cc3ccccc3)C(=O)NCc4ccccc4"]
ligand_folder = 'Ligands'

long_ligand_list = []
for i in smilesstrings:
    long_ligand_list += [Chem.MolFromSmiles(i)]

for j, query in enumerate(long_ligand_list):
    writer = SDWriter('%s/Ligand_%s.sdf' %(ligand_folder, '{0:03}'.format(j)))
    writer.write(query)

# Make sure you run Fragmentise.py on those Ligands

## Load the FragmentKB and Probability.mat FFN output

In [ ]:
normalDF = pickle.load(open("../EnsembleModel/Data/GrandCID.dict", "rb"))

probabilitymat = pickle.load(open("Analysis/COVID19_coll_noEnv75.mat", "rb"))
probabilitymat = probabilitymat.reshape(75,59732) #row - environment, column - fragments

## Extract the predicted fragments

In [ ]:
# Extracts the Fragment indicices of Fragments with higher binding probability than the cut-off.

def extractHighProbFrags(env, probsMat, cut):
    envN = probsMat[env] # select all fragments in the relevant environment
    indx = np.where(envN>cut)
    return indx

# collects all indices per environment - TODO: group them by regional environments
highProbsFragsIndices = np.array([], dtype=int)

for i in range(probabilitymat.shape[0]):
    highProbsFragsIndices = np.append(highProbsFragsIndices, extractHighProbFrags(i, probabilitymat, 0.97))

unique_highProbsFragsIndices, counts_indicies = np.unique(highProbsFragsIndices, return_counts=True)

print("Average prediction per Environment:", highProbsFragsIndices.shape[0]//probabilitymat.shape[0])
print("Unique predictions across all", probabilitymat.shape[0], "Environments:" , unique_highProbsFragsIndices.shape[0])
print("Unique predictions predicted by more than 25% of the environments:", unique_highProbsFragsIndices[counts_indicies>probabilitymat.shape[0]//4].shape[0])
print("Unique predictions predicted by more than 50% of the environments:", unique_highProbsFragsIndices[counts_indicies>probabilitymat.shape[0]//2].shape[0])
print("Unique predictions predicted by more than 75% of the environments:", unique_highProbsFragsIndices[counts_indicies>probabilitymat.shape[0]//4*3].shape[0])

## Optional: Reduce Prediction Fragments based on occurance

In [ ]:
unique_highProbsFragsIndices = unique_highProbsFragsIndices[counts_indicies>probabilitymat.shape[0]//2]

## Visualise the predicted fragments

In [ ]:
predicted_molecules = normalDF.iloc[unique_highProbsFragsIndices]['Mol']
Draw.MolsToGridImage(predicted_molecules.values, molsPerRow=10, subImgSize=(150,150), maxMols=500)

## Cluster the predicted fragments into prediction clusters: Select number of clusters based on the dendogram - around 10 fragments per cluster on average

In [ ]:
def TanSim(indx1, indx2):
    b=AllChem.GetMorganFingerprintAsBitVect(predicted_molecules.iloc[indx1],2)
    c=AllChem.GetMorganFingerprintAsBitVect(predicted_molecules.iloc[indx2],2)
    return DataStructs.FingerprintSimilarity(b,c)

sim = np.zeros((predicted_molecules.shape[0],predicted_molecules.shape[0]))
for i in range(predicted_molecules.shape[0]):
    for j in range(predicted_molecules.shape[0]):
        sim[i,j] = TanSim(i,j)
        if i == j:
            break
sim = sim + sim.transpose() - np.eye(predicted_molecules.shape[0])

plt.figure(figsize=(15, 7))
plt.title("Customer Dendograms")
dend = shc.dendrogram(shc.linkage(sim, method='ward'))
#plt.savefig('Ligand_CV_Results/!Examples/dendogram.png')
plt.show()

## Actual Clustering using Agglomerative Clustering and save the clusters

In [ ]:
clustering = AgglomerativeClustering(n_clusters=11).fit(sim)
clbls = clustering.labels_

In [ ]:
if not os.path.isdir("%s/!PredictionFragments" %ligand_folder):
    os.makedirs("%s/!PredictionFragments" %ligand_folder)
for i in range(max(clbls)+1):
    img = Draw.MolsToGridImage(predicted_molecules.iloc[np.where(clbls==i)[0]].values, molsPerRow=10, subImgSize=(150,150))
    img.save('%s/!PredictionFragments/Cluster_%s.png' %(ligand_folder,'{0:02}'.format(i)))

# Cluster 5
Draw.MolsToGridImage(predicted_molecules.iloc[np.where(clbls==4)[0]].values, molsPerRow=10, subImgSize=(150,150))

## Mapping the predicted Fragments onto the Ligands

In [ ]:
def automateit(testname):
    # load the original molecule and save the image    
    print("%s" %(testname))
    suppl = Chem.SDMolSupplier('%s/%s.sdf' %(ligand_folder, testname))
    mols_op = [x for x in suppl]
    for m in mols_op: tmp=AllChem.Compute2DCoords(m)
    img = Draw.MolsToGridImage(mols_op, molsPerRow=1, subImgSize=(500,500))#AllChem.Compute2DCoords(suppl)
    img.save('%s/%s/%s.png' %(ligand_folder, testname, testname))
    
    # load the fragments of the molecule and save the image
    ligfrags = pickle.load(open("%s/%s/%s.list" %(ligand_folder,testname,testname), "rb"))
    frags = np.empty(len(ligfrags))
    for j, i in enumerate(ligfrags):
        frags[j] = normalDF.index.get_loc(i)
    fragmols = normalDF.iloc[frags]['Mol']
    img = Draw.MolsToGridImage(fragmols.values, molsPerRow=10, subImgSize=(150,150), maxMols=100)
    img.save('%s/%s/%s-Fragmentised.png' %(ligand_folder, testname, testname))
    print("Number of Fragmentised Fragments:", frags.shape[0])
    
    #find pockets 
    pocket_1 = range(75)

    
    correct_ident_frags = np.array([]) # Indices of all correctly identified fragments
    all_coll = np.array([]) # Indicies of all predicted fragments
    
    # for environment in query environments find the predicted fragments and collect them
    for i in pocket_1:
        all_pred = extractHighProbFrags(i, probabilitymat, 0.97)[0]
        all_coll = np.append(all_coll, all_pred)
    
    # find the unique fragments and keep fragments that show up in more than 25% of environments
    unique_highProbsFragsIndices, counts_indicies = np.unique(all_coll, return_counts=True)
    all_pred = unique_highProbsFragsIndices[counts_indicies>probabilitymat.shape[0]//2]
    
    # find the overlap between predicted fragments and ligand substructure fragments
    identified, identified_i, _ = np.intersect1d(all_pred, frags, return_indices=True)
    correct_ident_frags = np.append(correct_ident_frags, identified)

    print("Unique Fragments identified:", np.unique(correct_ident_frags).shape[0], "out of", frags.shape[0], "using", np.unique(all_pred).shape[0], "predictions")
    
    # find the unique clusters that have identified predictions
    clust_labels = clbls[np.intersect1d(all_pred, correct_ident_frags, return_indices=True)[1]]
    print(np.unique(clust_labels).shape[0], "unique clusters predicted out of", max(clbls)+1)
    
    #Find the total number of atoms covered
    total_atoms = np.array([])
    perc_atoms = np.array([])
    for k in correct_ident_frags:
        k = int(k)
        atom_substruct = mols_op[0].GetSubstructMatches(normalDF.iloc[k]['Mol'])
        total_atoms = np.append(total_atoms, atom_substruct[0])
        try:
            total_atoms = np.append(total_atoms, atom_substruct[1])
        except:
            pass
    total_atoms = np.unique(total_atoms)
    perc_atoms = np.append(perc_atoms, np.round(total_atoms.shape[0]/mols_op[0].GetNumAtoms()*100, decimals=0))
    print(np.round(total_atoms.shape[0]/mols_op[0].GetNumAtoms()*100, decimals=0) , "% Identified", total_atoms.shape[0], "Atoms out of", mols_op[0].GetNumAtoms())
    print("")
    return total_atoms.astype(int).tolist(), perc_atoms, np.unique(all_coll), np.unique(correct_ident_frags), clust_labels

testnames = [f[:-4] for f in os.listdir(ligand_folder) if f[-4:]==".sdf"]
sums = []
pc_atom = []
pred_frags = [] 
ident_frags = []
clusters = []
for i in testnames:
    a, b, c, d, e = automateit(i)
    sums = sums + [a]
    pc_atom += [b[0]]
    pred_frags += [c]
    ident_frags += [d]
    clusters += [e]

## Plot Atom Coverage across all ligands

In [ ]:
atom_coverage = np.array(pc_atom)
plt.hist(atom_coverage, bins=20)
plt.gca().set(title='Atom Percentage Covered in 105 Native Inhibitors', ylabel='Count', xlabel='Atom Percentage');

## Draw Molecules with Atom Indices

In [ ]:
testnames = [f[:-4] for f in os.listdir(ligand_folder) if f[-4:]==".sdf"]
def ligand_indx_draw(testname):
    suppl = Chem.SDMolSupplier('%s/%s.sdf' %(ligand_folder, testname))
    mols_op = [x for x in suppl]
    for m in mols_op: tmp=AllChem.Compute2DCoords(m)
    for mol in mols_op: 
        d = rdMolDraw2D.MolDraw2DCairo(800, 800) # or MolDraw2DSVG to get SVGs
        d.drawOptions().addAtomIndices = True
        d.DrawMolecule(mol)
        d.FinishDrawing()
        with open('%s/%s/%s_Indx.png' %(ligand_folder, testname, testname), 'wb') as f:
            f.write(d.GetDrawingText())
    
for i in testnames:
    ligand_indx_draw(i)

## Draw Atom Coverage

In [ ]:
def ligand_covered_draw(testname, atoms_cov):
    suppl = Chem.SDMolSupplier('%s/%s.sdf' %(ligand_folder, testname))
    mols_op = [x for x in suppl]
    hit_bonds = []
    for mol in mols_op:
        for i in range(mol.GetNumBonds()):
            a1 = mol.GetBondWithIdx(i).GetBeginAtomIdx()
            a2 = mol.GetBondWithIdx(i).GetEndAtomIdx()
            if (a1 in atoms_cov) and (a2 in atoms_cov):
                hit_bonds.append(i)

    colour = (0.2,1,0.2) #light green
    atom_cols = {}
    for i, at in enumerate(atoms_cov):
        atom_cols[at] = colour
    bond_cols = {}
    for i, bd in enumerate(hit_bonds):
        bond_cols[bd] = colour
    for m in mols_op: tmp=AllChem.Compute2DCoords(m)
    for mol in mols_op: 
        d = rdMolDraw2D.MolDraw2DCairo(800, 800) # or MolDraw2DSVG to get SVGs
        d.DrawMolecule(mol)
        d.FinishDrawing()
        rdMolDraw2D.PrepareAndDrawMolecule(d, mol, highlightAtoms=atoms_cov, highlightBonds=hit_bonds,highlightAtomColors=atom_cols, highlightBondColors=bond_cols)
        with open('%s/%s/%s_Cov.png' %(ligand_folder, testname, testname), 'wb') as f:
            f.write(d.GetDrawingText())
            
for j, i in enumerate(testnames):
    ligand_covered_draw(i, sums[j])